In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
from datetime import datetime, timedelta
import pandas as pd

current_date = pd.to_datetime(datetime.utcnow()).floor('H') # - timedelta(hours=1)
print(f'{current_date=}')
# current_date = pd.Timestamp('2025-02-16 00:00:00')

current_date=Timestamp('2025-02-16 16:00:00')


In [9]:
from src.inference import load_batch_of_features_from_store

features = load_batch_of_features_from_store(current_date)

2025-02-16 17:02:13,362 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-16 17:02:13,370 INFO: Initializing external client
2025-02-16 17:02:13,370 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-16 17:02:14,399 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213632
2025-02-16 17:02:16,691 INFO: Feature view already exists, skipping creation.
2025-02-16 17:02:16,691 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-16 17:02:16,699 INFO: Initializing external client
2025-02-16 17:02:16,706 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-16 17:02:17,772 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213632
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (11.42s) 


In [12]:
from src.inference import (
    load_model_from_registry,
    get_model_predictions
)

model = load_model_from_registry()
predictions = get_model_predictions(model, features)

2025-02-16 17:05:08,853 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-16 17:05:08,862 INFO: Initializing external client
2025-02-16 17:05:08,862 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-16 17:05:09,976 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213632
[LightGBM] [Warning] feature_fraction is set=0.6755137632255699, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6755137632255699
[LightGBM] [Warning] bagging_fraction is set=0.7317109108834197, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7317109108834197


In [13]:
predictions['pickup_hour'] = current_date
predictions

,pickup_location_id,predicted_demand,pickup_hour
0,1,3.0,2025-02-16 16:00:00
1,2,0.0,2025-02-16 16:00:00
2,3,0.0,2025-02-16 16:00:00
3,4,4.0,2025-02-16 16:00:00
4,5,0.0,2025-02-16 16:00:00
...,...,...,...
260,261,45.0,2025-02-16 16:00:00
261,262,59.0,2025-02-16 16:00:00
262,263,96.0,2025-02-16 16:00:00
263,264,19.0,2025-02-16 16:00:00


In [14]:
from src.feature_store_api import get_feature_store
import src.config as config

# connect to the feature group
feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTIONS,
    version=1,
    description="Predictions generate by our production model",
    primary_key = ['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
)

2025-02-16 17:05:42,572 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-16 17:05:42,572 INFO: Initializing external client
2025-02-16 17:05:42,572 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-16 17:05:43,797 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213632


In [15]:
feature_group.insert(predictions, write_options={"wait_for_job": False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1213632/fs/1201257/fg/1399084


Uploading Dataframe: 100.00% |██████████| Rows 265/265 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: model_predictions_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1213632/jobs/named/model_predictions_feature_group_1_offline_fg_materialization/executions


(Job('model_predictions_feature_group_1_offline_fg_materialization', 'SPARK'),
 None)